In [1]:
pip install nltk
pip inst

In [2]:
pip install scikit-learn # Helps in Classification

In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity # To check two movies are similar or not

In [5]:
netflix_df = pd.read_csv("/content/netflixData.csv")
print(netflix_df.head())

                                Show Id                          Title  \
0  cc1b6ed9-cf9e-4057-8303-34577fb54477                       (Un)Well   
1  e2ef4e91-fb25-42ab-b485-be8e3b23dedb                         #Alive   
2  b01b73b7-81f6-47a7-86d8-acb63080d525  #AnneFrank - Parallel Stories   
3  b6611af0-f53c-4a08-9ffa-9716dc57eb9c                       #blackAF   
4  7f2d4170-bab8-4d75-adc2-197f7124c070               #cats_the_mewvie   

                                         Description  \
0  This docuseries takes a deep dive into the luc...   
1  As a grisly virus rampages a city, a lone man ...   
2  Through her diary, Anne Frank's story is retol...   
3  Kenya Barris and his family navigate relations...   
4  This pawesome documentary explores how our fel...   

                      Director  \
0                          NaN   
1                       Cho Il   
2  Sabina Fedeli, Anna Migotto   
3                          NaN   
4             Michael Margolis   

             

In [6]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Show Id             5967 non-null   object 
 1   Title               5967 non-null   object 
 2   Description         5967 non-null   object 
 3   Director            3903 non-null   object 
 4   Genres              5967 non-null   object 
 5   Cast                5437 non-null   object 
 6   Production Country  5408 non-null   object 
 7   Release Date        5964 non-null   float64
 8   Rating              5963 non-null   object 
 9   Duration            5964 non-null   object 
 10  Imdb Score          5359 non-null   object 
 11  Content Type        5967 non-null   object 
 12  Date Added          4632 non-null   object 
dtypes: float64(1), object(12)
memory usage: 606.1+ KB


In [7]:
print(netflix_df.isnull())

      Show Id  Title  Description  Director  Genres   Cast  \
0       False  False        False      True   False   True   
1       False  False        False     False   False  False   
2       False  False        False     False   False  False   
3       False  False        False      True   False  False   
4       False  False        False     False   False   True   
...       ...    ...          ...       ...     ...    ...   
5962    False  False        False     False   False  False   
5963    False  False        False      True   False  False   
5964    False  False        False      True   False  False   
5965    False  False        False     False   False  False   
5966    False  False        False     False   False  False   

      Production Country  Release Date  Rating  Duration  Imdb Score  \
0                  False         False   False     False       False   
1                  False         False   False     False       False   
2                  False         False 

In [8]:
print(netflix_df.isnull().sum())

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


In [9]:
required_nf_df = netflix_df[["Title", "Description", "Content Type", "Genres"]]
required_nf_df.head()

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"


In [10]:
# Dropping the columns with Null Values
required_nf_df.dropna()

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"
...,...,...,...,...
5962,الف مبروك,"On his wedding day, an arrogant, greedy accoun...",Movie,"Comedies, Dramas, International Movies"
5963,دفعة القاهرة,A group of women leaves Kuwait to attend unive...,TV Show,"International TV Shows, TV Dramas"
5964,海的儿子,"Two brothers start a new life in Singapore, wh...",TV Show,"International TV Shows, TV Dramas"
5965,반드시 잡는다,After people in his town start turning up dead...,Movie,"Dramas, International Movies, Thrillers"


In [11]:
required_nf_df.count()

Title           5967
Description     5967
Content Type    5967
Genres          5967
dtype: int64

In [17]:
# Cleaning the Data
import nltk
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english") # Covertin the long words into the Root word.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
from nltk.corpus import stopwords
import string
stopword = set(stopwords.words('english'))

In [19]:
import re # Regular Expression

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [20]:
required_nf_df['Title'] = required_nf_df['Title'].apply(clean)

<ipython-input-20-21c0b3ed5d0d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_nf_df['Title'] = required_nf_df['Title'].apply(clean)


In [21]:
print(required_nf_df['Title'])

0                           unwel
1                            aliv
2       annefrank  parallel stori
3                         blackaf
4                    catsthemewvi
                  ...            
5962                    الف مبروك
5963                 دفعة القاهرة
5964                         海的儿子
5965                      반드시 잡는다
5966           최강전사 미니특공대  영웅의 탄생
Name: Title, Length: 5967, dtype: object


In [22]:
genreList = required_nf_df['Genres'].tolist()
tfidf = text.TfidfVectorizer(stop_words = "english")
tfidf_matrix = tfidf.fit_transform(genreList)
similarity = cosine_similarity(tfidf_matrix) # We find the Cosine angle between the words and if the angle comes out to be 0 degrees then the words are similar

In [23]:
indices = pd.Series(required_nf_df.index, index= required_nf_df["Title"]).drop_duplicates()

In [25]:
# Algorithm
def netflix_recommendation(title, similarity=similarity):
  index = indices[title]
  similarity_score = list(enumerate(similarity[index]))
  similarity_score = similarity_score[0:10]
  movie_indices=[i[0] for i in similarity_score]
  return required_nf_df['Title'].iloc[movie_indices]

In [26]:
print(netflix_recommendation('fitoor'))

0                        unwel
1                         aliv
2    annefrank  parallel stori
3                      blackaf
4                 catsthemewvi
5               friendbutmarri
6              friendbutmarri 
7                  realityhigh
8                             
9                        selfi
Name: Title, dtype: object
